In [1]:
from embedding_helper import *
from data_handler import *

In [2]:
DIMS = 50
CUTOFF = 0.00002

In [3]:
# loading embeddings
entity_vec, relations_vec = load_embeddings(dims=DIMS)
entity_dict, relations_dict = load_ids()

male_vector, female_vector = get_gender_vectors(entity_vec, entity_dict, DIMS)

# getting occupation sets
triplets, entity_labels, property_labels = load_data()
male_occupations, female_occupations, neutral_occupations = get_male_female_neutral_occupations(triplets, entity_labels, property_labels, CUTOFF)



In [4]:
def get_occupation_projection_score(occupation_list):
    # here we skip the occupations whose vector is not available
    gender_vector = (female_vector - male_vector)/np.linalg.norm(female_vector - male_vector)
    projection_score = 0
    count = 0
    for i in occupation_list:
        occupation_id = get_entity_id(entity_labels, i)
        occupation_id_with_vector = []
        for occ_id in occupation_id:
            if occ_id in entity_dict:
                occupation_id_with_vector.append(occ_id)
        if len(occupation_id_with_vector)==0:
            continue
        occupation_vector = get_averaged_entity_vector(entity_vec, entity_dict,DIMS,occupation_id_with_vector)
        projection = np.dot(gender_vector.transpose(), occupation_vector)
        projection_score += projection
        count += 1
    return projection_score/count


In [5]:
# first row of Table III
male_occupation_score = get_occupation_projection_score(male_occupations)
female_occupation_score = get_occupation_projection_score(female_occupations)
neutral_occupation_score = get_occupation_projection_score(neutral_occupations)
print("Projection Score\tMale\t\tFemale\t\tNeutral")
print("\t\t\t%f\t%f\t%f"%(male_occupation_score, female_occupation_score, neutral_occupation_score))

Projection Score	Male		Female		Neutral
			0.177166	0.155831	0.104418
